<a href="https://colab.research.google.com/github/LuizzOliveira/Desafios_aprendizagem/blob/main/Estrat%C3%A9giaCantos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests fake_useragent pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 3.9 MB/s eta 0:00:00


### Instalando as Bibliotecas

### Importando as Bibliotecas

In [ ]:
import requests
from fake_useragent import UserAgent
import telebot
import pytz
import time
import atexit
from datetime import datetime, timedelta
from requests.exceptions import RequestException

### Configuraçoes do bot

In [ ]:
bot = telebot.TeleBot(telegram_token)
text = "<b>Bot Ligado!!</b> 🚀"
bot.send_message(chat_id=chat_id, text=text, parse_mode="html", disable_web_page_preview=True)

jogos_enviados = set()

def obter_dados():
    try:
        ua = UserAgent()
        headers = {
            'User-Agent': ua.random,
        }
        url = 'https://api.sportsanalytics.com.br/api/v1/fixtures-svc/fixtures/livescores'
        params = {
            'include': 'weatherReport,additionalInfo,league,stats,pressureStats,probabilities'
        }
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        dados = response.json()
        return dados
    except RequestException as e:
        print(f"Erro ao fazer a requisição: {e}")
        return None

def obter_caracteristicas(dados):
    caracteristicas = [
        {
            'fixtureId': jogo['fixtureId'],
            'timeC': jogo['homeTeam'].get('name', ''),
            'timeV': jogo['awayTeam'].get('name', ''),
            'placarC': jogo['scores'].get('homeTeamScore', 0),
            'placarV': jogo['scores'].get('awayTeamScore', 0),
            'liga': jogo['league'].get('name', ''),
            'minuto': jogo['currentTime'].get('minute', 0),

            'ataques_perigosos_C': jogo.get('stats', {}).get('dangerousAttacks', {}).get('home', 0) if jogo.get('stats') and jogo.get('stats', {}).get('dangerousAttacks') else 0,
            'ataques_perigosos_V': jogo.get('stats', {}).get('dangerousAttacks', {}).get('away', 0) if jogo.get('stats') and jogo.get('stats', {}).get('dangerousAttacks') else 0,
            'chutes_no_gol_C': jogo.get('stats', {}).get('shotsOngoal', {}).get('home', 0) if jogo.get('stats') and jogo.get('stats', {}).get('shotsOngoal') else 0,
            'chutes_no_gol_V': jogo.get('stats', {}).get('shotsOngoal', {}).get('away', 0) if jogo.get('stats') and jogo.get('stats', {}).get('shotsOngoal') else 0,
            'chutes_fora_C': jogo.get('stats', {}).get('shotsOffgoal', {}).get('home', 0) if jogo.get('stats') and jogo.get('stats', {}).get('shotsOffgoal') else 0,
            'chutes_fora_V': jogo.get('stats', {}).get('shotsOffgoal', {}).get('away', 0) if jogo.get('stats') and jogo.get('stats', {}).get('shotsOffgoal') else 0,
            'escanteios_C': jogo.get('stats', {}).get('corners', {}).get('home', 0) if jogo.get('stats') and jogo.get('stats', {}).get('corners') else 0,
            'escanteios_V': jogo.get('stats', {}).get('corners', {}).get('away', 0) if jogo.get('stats') and jogo.get('stats', {}).get('corners') else 0
        }
        for jogo in (j for j in dados['data'] if j and isinstance(j, dict))
    ]
    return caracteristicas

def estrategia(caracteristicas, nome_estrategia, nome_bot, minutos_min, minutos_max, total_chutes, total_gols, total_cantos, total_ataques_perigosos, total_ataques_perigosos_pm):
    count = 0
    for jogo in (j for j in caracteristicas if minutos_min <= j['minuto'] <= minutos_max):
        jogo_id = jogo['fixtureId']
        if jogo_id not in jogos_enviados:
            shots = jogo['chutes_no_gol_C'] + jogo['chutes_no_gol_V'] + jogo['chutes_fora_C'] + jogo['chutes_fora_V']
            if shots >= total_chutes and jogo['placarC'] + jogo['placarV'] >= total_gols and jogo['escanteios_C'] + jogo['escanteios_V'] >= total_cantos and jogo['ataques_perigosos_C'] + jogo['ataques_perigosos_V'] >= total_ataques_perigosos:
                appmT = (jogo['ataques_perigosos_C'] + jogo['ataques_perigosos_V']) / jogo['minuto']
if appmT >= total_ataques_perigosos_pm:
                    convert_nome = jogo['timeC'].replace(" ","+")
                    link_bet365 = f"https://www.bet365.com/#/AX/{convert_nome}/"
                    count += 1
                    text = f"Estratégia:"
<b>{nome_estrategia}</b>

🆚 <b>{jogo['timeC']} x {jogo['timeV']}</b>
🏆 {jogo['liga']}
⏰ {jogo['minuto']}' minutos

📲 <a href='{link_bet365}'><b>Link do jogo na Bet365</b></a>

📊<b>Estatísticas (Casa - Fora)</b>
🥅 Placar: {jogo['placarC']} - {jogo['placarV']}
🔥 Ataques Perigosos: {jogo['ataques_perigosos_C']} - {jogo['ataques_perigosos_V']}
🎯 Chutes ao gol: {jogo['chutes_no_gol_C']} - {jogo['chutes_no_gol_V']}
💥 Chutes fora do gol: {jogo['chutes_fora_C']} - {jogo['chutes_fora_V']}
⛳️ Escanteios: {jogo['escanteios_C']} - {jogo['escanteios_V']}

📊 <b>{nome_bot}</b> analisou as estatísticas e aprovou!

<b>Pré-live</b> 🆗  <b>Ao-vivo</b> 🆗'''
                    bot.send_message(chat_id=chat_id, text=text, parse_mode="html", disable_web_page_preview=True)
                    jogos_enviados.add(jogo_id)
    return count

while True:
    try:
        dados = obter_dados()
        caracteristicas = obter_caracteristicas(dados)
        estrategia(caracteristicas, nome_estrategia, nome_bot, minutos_min, minutos_max, total_chutes, total_gols, total_cantos, total_ataques_perigosos, total_ataques_perigosos_pm)
    except RequestException as e:
        print(f"Erro ao fazer a requisição: {e}")
    time.sleep(120)

SyntaxError: invalid character '🆚' (U+1F19A) (<ipython-input-9-83dfc365cf9d>, line 64)

### Corpo do Bot

In [ ]:
# Configurações do bot do Telegram
telegram_token = "6135732046:AAEVXxOwqGU3dV4q94qQCHK622t_jAWCq6A"
chat_id = "-882967126"

# Configurações da estratégia
nome_estrategia = "cantos"
nome_bot = "SoccerBot"
minutos_min = 85 # Tempo mínimo do jogo para envio de alerta
minutos_max = 87 # Tempo máximo do jogo para envio de alerta
total_gols = 0 # Número máximo total de gols
total_chutes = 0 # Total de chutes, independentemente de serem no alvo ou não
total_cantos = 8 # Número mínimo total de cantos
total_ataques_perigosos = 0 # Número mínimo total de ataques perigosos
total_ataques_perigosos_pm = 0 # Número mínimo total de ataques perigosos por minuto para ambas as equipes